# SQLAlchemy

## What is SQLAlchemy?

From the website:

> SQLAlchemy is the Python SQL toolkit and Object Relational Mapper that gives application developers the full power and flexibility of SQL.

## Expand Your Acronyms

### SQL

Structured Query Language. Domain specific languate used to manage data in an RDBMS.  There is an ANSI standard, but pretty much all databases do their own thing.

### RDBMS

Relational database management system.  For example SQLite, PostgreSQL, MariaDB (MySQL), etc.

## Using the SQLAlchemy ORM

SQLAlchemy consists of two components:  SQLAlchemy Core and the SQLAlchemy ORM.

### Version Check

In [1]:
import sys
print(sys.version)

3.6.0 (default, Jan  4 2017, 14:51:27) 
[GCC 4.2.1 Compatible Apple LLVM 8.0.0 (clang-800.0.42.1)]


In [2]:
import sqlalchemy
print(sqlalchemy.__version__)

1.1.4


### Create an Engine

In SQLAlchemy the Engine is the starting point for any application.  To create an engine you'll need a [Database URL](http://docs.sqlalchemy.org/en/rel_1_1/core/engines.html#database-urls) (aka connection string).

In [3]:
import os
from sqlalchemy import create_engine

engine = create_engine('sqlite:///:memory:', echo=True)

# engine = create_engine('sqlite:///{!s}/data.sqlite3'.format(os.getcwd()))

### Declaring a Mapping

SQLAlchemy ORM uses a system known as *Declarative* to create Python classes and describe the database that they will be mapped to.

In [4]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

In [5]:
from sqlalchemy import Column, Date, Integer, String

class Author(Base):
    __tablename__ = 'authors'
    
    id = Column(Integer, primary_key=True)
    first_name = Column(String)
    last_name = Column(String, nullable=False)
    birthdate = Column(Date)
    
    def __repr__(self):
        return 'Author(first_name={!r}, last_name={!r}, birthdate={!r})'.\
            format(self.first_name, self.last_name, self.birthdate)

### Creating the Schema

In [6]:
Base.metadata.create_all(engine)

2017-01-05 19:21:27,893 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-01-05 19:21:27,895 INFO sqlalchemy.engine.base.Engine ()
2017-01-05 19:21:27,896 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-01-05 19:21:27,897 INFO sqlalchemy.engine.base.Engine ()
2017-01-05 19:21:27,898 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("authors")
2017-01-05 19:21:27,899 INFO sqlalchemy.engine.base.Engine ()
2017-01-05 19:21:27,901 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE authors (
	id INTEGER NOT NULL, 
	first_name VARCHAR, 
	last_name VARCHAR NOT NULL, 
	birthdate DATE, 
	PRIMARY KEY (id)
)


2017-01-05 19:21:27,901 INFO sqlalchemy.engine.base.Engine ()
2017-01-05 19:21:27,902 INFO sqlalchemy.engine.base.Engine COMMIT


### Creating instances of mapped classes

In [8]:
from datetime import date

chuck = Author(first_name='Chuck', 
               last_name='Palahniuk', 
               birthdate=date(1962, 2, 21))

In [12]:
chuck.id is None

True

### Sessions

SQLAlchemy docs call this the ORM's "handle" to the database.

In [13]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)

To save an instance of a mapped class we add it to an instance of a session

In [14]:
session = Session()
session.add(chuck)

At this point the instance is __pending__.  The session will issue the SQL when it's needed using a process known as __flush__.

In [15]:
chuck.id is None

True

In [16]:
session.new

IdentitySet([Author(first_name='Chuck', last_name='Palahniuk', birthdate=datetime.date(1962, 2, 21))])

Sessions sometimes flush automatically, for example when issuing a query:

In [17]:
queried_author = session.query(Author).\
    filter_by(first_name='Chuck').first()

2017-01-05 19:27:43,960 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-01-05 19:27:43,964 INFO sqlalchemy.engine.base.Engine INSERT INTO authors (first_name, last_name, birthdate) VALUES (?, ?, ?)
2017-01-05 19:27:43,965 INFO sqlalchemy.engine.base.Engine ('Chuck', 'Palahniuk', '1962-02-21')
2017-01-05 19:27:43,966 INFO sqlalchemy.engine.base.Engine SELECT authors.id AS authors_id, authors.first_name AS authors_first_name, authors.last_name AS authors_last_name, authors.birthdate AS authors_birthdate 
FROM authors 
WHERE authors.first_name = ?
 LIMIT ? OFFSET ?
2017-01-05 19:27:43,967 INFO sqlalchemy.engine.base.Engine ('Chuck', 1, 0)


In [18]:
chuck.id

1

In [19]:
queried_author is chuck

True

In [20]:
session.add_all([
    Author(first_name='John', last_name='Steinbeck', birthdate=date(1902, 2, 27)),
    Author(first_name='Philip', last_name='Dick', birthdate=date(1928, 12, 16)),
    Author(first_name='Douglas', last_name='Adams', birthdate=date(1952, 3, 11))
])

In [21]:
session.new

IdentitySet([Author(first_name='John', last_name='Steinbeck', birthdate=datetime.date(1902, 2, 27)), Author(first_name='Philip', last_name='Dick', birthdate=datetime.date(1928, 12, 16)), Author(first_name='Douglas', last_name='Adams', birthdate=datetime.date(1952, 3, 11))])

Updating instances

In [22]:
chuck.first_name = 'Charles'

In [23]:
session.dirty

IdentitySet([Author(first_name='Charles', last_name='Palahniuk', birthdate=datetime.date(1962, 2, 21))])

In [24]:
session.commit()

2017-01-05 19:29:54,791 INFO sqlalchemy.engine.base.Engine UPDATE authors SET first_name=? WHERE authors.id = ?
2017-01-05 19:29:54,793 INFO sqlalchemy.engine.base.Engine ('Charles', 1)
2017-01-05 19:29:54,794 INFO sqlalchemy.engine.base.Engine INSERT INTO authors (first_name, last_name, birthdate) VALUES (?, ?, ?)
2017-01-05 19:29:54,795 INFO sqlalchemy.engine.base.Engine ('John', 'Steinbeck', '1902-02-27')
2017-01-05 19:29:54,796 INFO sqlalchemy.engine.base.Engine INSERT INTO authors (first_name, last_name, birthdate) VALUES (?, ?, ?)
2017-01-05 19:29:54,796 INFO sqlalchemy.engine.base.Engine ('Philip', 'Dick', '1928-12-16')
2017-01-05 19:29:54,798 INFO sqlalchemy.engine.base.Engine INSERT INTO authors (first_name, last_name, birthdate) VALUES (?, ?, ?)
2017-01-05 19:29:54,798 INFO sqlalchemy.engine.base.Engine ('Douglas', 'Adams', '1952-03-11')
2017-01-05 19:29:54,800 INFO sqlalchemy.engine.base.Engine COMMIT


Sessions can also be rolled back to undo any changes that have not been commited.

In [25]:
poe = Author(first_name='Edgar', last_name='Poe', birthdate=date(1809, 1, 19))
session.add(poe)

In [26]:
chuck.first_name = 'Something Silly'

In [27]:
session.rollback()

In [28]:
chuck.first_name

2017-01-05 19:31:38,114 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-01-05 19:31:38,116 INFO sqlalchemy.engine.base.Engine SELECT authors.id AS authors_id, authors.first_name AS authors_first_name, authors.last_name AS authors_last_name, authors.birthdate AS authors_birthdate 
FROM authors 
WHERE authors.id = ?
2017-01-05 19:31:38,117 INFO sqlalchemy.engine.base.Engine (1,)


'Charles'

In [29]:
poe in session

False

### Querying

You can query using Classes:

In [31]:
for instance in session.query(Author).order_by(Author.last_name):
    print(instance)

2017-01-05 19:32:52,618 INFO sqlalchemy.engine.base.Engine SELECT authors.id AS authors_id, authors.first_name AS authors_first_name, authors.last_name AS authors_last_name, authors.birthdate AS authors_birthdate 
FROM authors ORDER BY authors.last_name
2017-01-05 19:32:52,620 INFO sqlalchemy.engine.base.Engine ()
Author(first_name='Douglas', last_name='Adams', birthdate=datetime.date(1952, 3, 11))
Author(first_name='Philip', last_name='Dick', birthdate=datetime.date(1928, 12, 16))
Author(first_name='Charles', last_name='Palahniuk', birthdate=datetime.date(1962, 2, 21))
Author(first_name='John', last_name='Steinbeck', birthdate=datetime.date(1902, 2, 27))


You can query using ORM-Instrumented descriptors:

In [32]:
for first, last in session.query(Author.first_name, Author.last_name):
    print(first, last)

2017-01-05 19:34:35,212 INFO sqlalchemy.engine.base.Engine SELECT authors.first_name AS authors_first_name, authors.last_name AS authors_last_name 
FROM authors
2017-01-05 19:34:35,214 INFO sqlalchemy.engine.base.Engine ()
Charles Palahniuk
John Steinbeck
Philip Dick
Douglas Adams


In [35]:
session.query(Author).filter(Author.last_name == 'Adams').one()

2017-01-05 19:35:55,596 INFO sqlalchemy.engine.base.Engine SELECT authors.id AS authors_id, authors.first_name AS authors_first_name, authors.last_name AS authors_last_name, authors.birthdate AS authors_birthdate 
FROM authors 
WHERE authors.last_name = ?
2017-01-05 19:35:55,597 INFO sqlalchemy.engine.base.Engine ('Adams',)


Author(first_name='Douglas', last_name='Adams', birthdate=datetime.date(1952, 3, 11))

In [39]:
session.query(Author).filter(Author.last_name.like('%ck')).\
    order_by(Author.last_name).all()

2017-01-05 19:38:04,760 INFO sqlalchemy.engine.base.Engine SELECT authors.id AS authors_id, authors.first_name AS authors_first_name, authors.last_name AS authors_last_name, authors.birthdate AS authors_birthdate 
FROM authors 
WHERE authors.last_name LIKE ? ORDER BY authors.last_name
2017-01-05 19:38:04,762 INFO sqlalchemy.engine.base.Engine ('%ck',)


[Author(first_name='Philip', last_name='Dick', birthdate=datetime.date(1928, 12, 16)),
 Author(first_name='John', last_name='Steinbeck', birthdate=datetime.date(1902, 2, 27))]

### Building relationships

In [40]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

class Book(Base):
    __tablename__ = 'books'
    
    id = Column(Integer, primary_key=True)
    title = Column(String, nullable=False)
    author_id = Column(Integer, ForeignKey('authors.id'))
    
    author = relationship("Author", back_populates="books")
    
    def __repr__(self):
        return 'Book(title={!r})'.format(self.title)
    
Author.books = relationship('Book', order_by=Book.id, 
                            back_populates='author')

In [41]:
Base.metadata.create_all(engine)

2017-01-05 19:41:51,060 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("authors")
2017-01-05 19:41:51,062 INFO sqlalchemy.engine.base.Engine ()
2017-01-05 19:41:51,063 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("books")
2017-01-05 19:41:51,064 INFO sqlalchemy.engine.base.Engine ()
2017-01-05 19:41:51,065 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE books (
	id INTEGER NOT NULL, 
	title VARCHAR NOT NULL, 
	author_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(author_id) REFERENCES authors (id)
)


2017-01-05 19:41:51,066 INFO sqlalchemy.engine.base.Engine ()
2017-01-05 19:41:51,067 INFO sqlalchemy.engine.base.Engine COMMIT


In [42]:
bram = Author(first_name='Bram', 
              last_name='Stoker', 
              birthdate=date(1847, 11, 8))

In [43]:
bram.books

[]

In [44]:
bram.books = [Book(title='Dracula')]

In [50]:
the_book = bram.books[0]
the_book.author

Author(first_name='Bram', last_name='Stoker', birthdate=datetime.date(1847, 11, 8))

In [51]:
session.add(bram)
session.commit()

2017-01-05 19:44:43,456 INFO sqlalchemy.engine.base.Engine INSERT INTO authors (first_name, last_name, birthdate) VALUES (?, ?, ?)
2017-01-05 19:44:43,457 INFO sqlalchemy.engine.base.Engine ('Bram', 'Stoker', '1847-11-08')
2017-01-05 19:44:43,460 INFO sqlalchemy.engine.base.Engine INSERT INTO books (title, author_id) VALUES (?, ?)
2017-01-05 19:44:43,460 INFO sqlalchemy.engine.base.Engine ('Dracula', 5)
2017-01-05 19:44:43,462 INFO sqlalchemy.engine.base.Engine COMMIT


In [52]:
bram = session.query(Author).filter_by(first_name='Bram').one()

2017-01-05 19:45:04,328 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-01-05 19:45:04,329 INFO sqlalchemy.engine.base.Engine SELECT authors.id AS authors_id, authors.first_name AS authors_first_name, authors.last_name AS authors_last_name, authors.birthdate AS authors_birthdate 
FROM authors 
WHERE authors.first_name = ?
2017-01-05 19:45:04,330 INFO sqlalchemy.engine.base.Engine ('Bram',)


In [59]:
session.query(Author, Book).filter(Author.id == Book.author_id).all()

2017-01-05 19:53:35,193 INFO sqlalchemy.engine.base.Engine SELECT authors.id AS authors_id, authors.first_name AS authors_first_name, authors.last_name AS authors_last_name, authors.birthdate AS authors_birthdate, books.id AS books_id, books.title AS books_title, books.author_id AS books_author_id 
FROM authors, books 
WHERE authors.id = books.author_id
2017-01-05 19:53:35,194 INFO sqlalchemy.engine.base.Engine ()


[(Author(first_name='Bram', last_name='Stoker', birthdate=datetime.date(1847, 11, 8)),
  Book(title='Dracula'))]

In [ ]:
session.query(Author)